In [ ]:
import os
import keras
import glob
import shutil
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import pathlib

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
AUTOTUNE = tf.data.experimental.AUTOTUNE
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
#dataset parameters
MODE = 'folder'
DATASET_PATH = '/Users/user_name/Desktop/New folder/Study/flower_photos'

In [ ]:
#image parameters
N_CLASSES = 5
IMG_SHAPE = 150
CHANNELS = 3
BATCH_SIZE = 100

In [ ]:
def group_images(dataset_path):
    imagepaths, labels = list(), list()
    try :
        classes = sorted(os.walk(dataset_path).__next__()[1])
    except Exception:
        classes = sorted(os.walk(dataset_path).__next__()[1])
    for c in classes:
        labels.append(c)
        c_dir = os.path.join(dataset_path,c)
        images = glob.glob(c_dir + '/*.jpg')
        #print("{}: {} Images".format(cl, len(images)))
        train, val = images[:round(len(images)*0.8)], images[round(len(images)*0.8):]
            
        for t in train:
            if not os.path.exists(os.path.join(dataset_path, 'train', c)):
                os.makedirs(os.path.join(dataset_path, 'train', c))
            shutil.move(t, os.path.join(dataset_path, 'train', c))

        for v in val:
            if not os.path.exists(os.path.join(dataset_path, 'val', c)):
                os.makedirs(os.path.join(dataset_path, 'val', c))
            shutil.move(v, os.path.join(dataset_path, 'val', c))
    

In [ ]:
# reading the dataset - two modes, file or folder
def read_images(dataset_path, mode, batch_size):
    
    imagepaths, labels = list(), list()
    if mode == 'file':
        with open(dataset_path) as f:
            data = f.read().splitlines()
        for d in data:
            imagepaths.append(d.split(' ')[0])
            labels.append(d.split(' ')[1])
    elif mode == 'folder':
        label=0
        try :
            classes = sorted(os.walk(dataset_path).__next__()[1])
        except Exception:
            classes = sorted(os.walk(dataset_path).__next__()[1])
        for c in classes:
            c_dir = os.path.join(dataset_path,c)
            try:
                walk = os.walk(c_dir).__next__()
            except:
                walk = os.walk(c_dir).__next__()
            for sample in walk[2]:
                if sample.endswith('.jpg') or sample.endswith('.jpeg'):
                    imagepaths.append(os.path.join(c_dir,sample))
                    #labels.append(label)
            #label = label+1
    else:
        raise Exception("Unknown mode")
        
    
    #converting to tensor
    imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    #labels = tf.convert_to_tensor(label, dtype=tf.string)
    """
    image, label = tf.train.slice_input_producer([imagepaths, labels], shuffle=True)
    
    #reading images in tf
    image = tf.read_file(image)
    image = tf.image.decode_jpeg(image, channels = CHANNELS)
    image = tf.image.resize_images(image,IMG_SHAPE)
    X, Y = tf.train.batch([image, label], batch_size=BATCH_SIZE,)
    """
    return imagepaths

In [ ]:
train_dir = os.path.join(DATASET_PATH,'train')
val_dir = os.path.join(DATASET_PATH,'val')
group_images(DATASET_PATH)
#X = read_images(train_dir,'folder',150)
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

In [ ]:
train_daisy_dir = os.path.join(train_dir, 'daisy')  # directory with our training cat pictures
train_dandelion_dir = os.path.join(train_dir, 'dandelion')  # directory with our training dog pictures
train_roses_dir = os.path.join(train_dir, 'roses')  # directory with our training cat pictures
train_sunflowers_dir = os.path.join(train_dir, 'sunflowers')  # directory with our training dog pictures
train_tulips_dir = os.path.join(train_dir, 'tulips')  # directory with our training cat pictures

val_daisy_dir = os.path.join(val_dir, 'daisy')  # directory with our validation cat pictures
val_dandelion_dir = os.path.join(val_dir, 'dandelion')  # directory with our validation dog pictures
val_roses_dir = os.path.join(val_dir, 'roses')  # directory with our validation dog pictures
val_sunflowers_dir = os.path.join(val_dir, 'sunflowers')  # directory with our validation dog pictures
val_tulips_dir = os.path.join(val_dir, 'tulips')  # directory with our validation dog pictures

total_train = len(os.listdir(train_daisy_dir)) +len(os.listdir(train_dandelion_dir)) +len(os.listdir(train_roses_dir)) +len(os.listdir(train_sunflowers_dir)) +len(os.listdir(train_tulips_dir))

total_val = len(os.listdir(val_daisy_dir))+len(os.listdir(val_dandelion_dir))+len(os.listdir(val_roses_dir))+len(os.listdir(val_sunflowers_dir))+len(os.listdir(val_tulips_dir))

print (total_train,'\t', total_val)

In [ ]:
image_gen_train = ImageDataGenerator(rescale=1./255,
                                     rotation_range=45,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                    )
train_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_SHAPE,IMG_SHAPE), 
                                                     class_mode='sparse'
                                                    )

image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=val_dir,
                                                 target_size=(IMG_SHAPE,IMG_SHAPE),
                                                 class_mode='sparse'
                                                )

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation ='relu', input_shape=(IMG_SHAPE,IMG_SHAPE,CHANNELS)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    tf.keras.layers.Conv2D(32, 3, padding='same', activation ='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    tf.keras.layers.Conv2D(64, 3, padding='same', activation ='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5,activation = 'softmax')
])

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs= 30
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()